In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Extract Sentiment
zip_path = '/content/drive/My Drive/AAI511_ML/midi_classic_music.zip'
extract_path = '/content/midi_classic_music/'

# Unzip the file.
!unzip -q -n "{zip_path}" -d "{extract_path}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Keep only the composers mentioned in the final project

Please only do your prediction only for below composers, therefore you need to select the required composers from the given dataset above.

1-Bach

2-Beethoven

3-Chopin

4-Mozart

In [11]:
import shutil

composers = ['Bach', 'Beethoven', 'Chopin', 'Mozart']

for item in os.listdir(extract_path):
    item_path = os.path.join(extract_path, item)
    if os.path.isfile(item_path) and (item.endswith('.mid') or item.endswith('.MID')):
        os.remove(item_path)
        print(f"Remove file at root: {item}")

midi_classic_path = extract_path + 'midiclassics'

for item in os.listdir(midi_classic_path):
    item_path = os.path.join(midi_classic_path, item)
    if os.path.isfile(item_path) and (item.endswith('.mid') or item.endswith('.MID')):
        os.remove(item_path)
        print(f"Remove file at midi_classic_path: {item}")

for root, dirs, files in os.walk(extract_path, topdown=False):
    for dir_name in dirs:
        # Check if this directory is one of our desired composers
        if dir_name in composers:
            continue  # Keep this directory

        # Check if this directory contains any of our desired composers as subdirectories
        full_path = os.path.join(root, dir_name)
        contains_desired = False

        for subroot, subdirs, subfiles in os.walk(full_path):
            if any(composer in subdirs for composer in composers):
                contains_desired = True
                break

        # If it doesn't contain desired composers, remove it
        if not contains_desired:
            shutil.rmtree(full_path)
            print(f"Removed: {full_path}")

Remove file at midi_classic_path: Tchaikovsky Lake Of The Swans Act 1 6mov.mid
Remove file at midi_classic_path: Diabelli Sonatina op151 n2 3mov.mid
Remove file at midi_classic_path: Rothchild Piano Sonata Rmw13 1mov.mid
Remove file at midi_classic_path: Kuhlau Sonatina op20 n1.mid
Remove file at midi_classic_path: Diabelli Sonatina op151 n3 2mov.mid
Remove file at midi_classic_path: Rothchild Horn Concerto Rmw16 1mov.mid
Remove file at midi_classic_path: Bizet Carmen Prelude.mid
Remove file at midi_classic_path: Tchaikovsky Lake Of The Swans Act 2 13mov.mid
Remove file at midi_classic_path: Buxethude Buxwv167 Canzonetta.mid
Remove file at midi_classic_path: Buxethude Buxwv136 Prelude.mid
Remove file at midi_classic_path: Rothchild Oboe Concerto Rmw09 3mov.mid
Remove file at midi_classic_path: Tchaikovsky Lake Of The Swans Act 1 9mov.mid
Remove file at midi_classic_path: Diabelli Sonatina op151 n2 2mov.mid
Remove file at midi_classic_path: Buxethude Buxwv156 Toccata.mid
Remove file at 